# Install Libraries

In [ ]:
# The exclamation mark (!) is used to run shell commands directly from Jupyter Notebook or Google Colab.
# This command installs the yt_dlp library, which is used for converting video to audio in Python.
!pip install yt_dlp

In [ ]:
# The exclamation mark (!) is used to run shell commands directly from Jupyter Notebook or Google Colab.
# This command installs the SpeechRecognition library, which is used for converting speech to text in Python.
!pip install speechrecognition

In [ ]:
# The exclamation mark (!) is used to run shell commands directly from Jupyter Notebook or Google Colab.
# This command installs the pydub library, which is used for audio processing tasks like converting, slicing, and merging audio files.
!pip install pydub

# Generating Transcript from Audio

In [ ]:
!pip install youtube-transcript-api

In [ ]:
!pip install pytube

In [ ]:
import re
import urllib.parse
import requests
from youtube_transcript_api import YouTubeTranscriptApi
from pytube import YouTube
import speech_recognition as sr
from pydub import AudioSegment
import os
import yt_dlp
def extract_video_id(video_url):
    """
    Extracts the YouTube video ID from various URL formats.
    """
    parsed_url = urllib.parse.urlparse(video_url)
    query_params = urllib.parse.parse_qs(parsed_url.query)

    if "v" in query_params:
        return query_params["v"][0]

    match = re.search(r"(youtu\.be/|youtube\.com/embed/|youtube\.com/shorts/)([\w-]+)", video_url)
    if match:
        return match.group(2)

    return None

def download_audio(video_url):
    """
    Downloads the audio using yt-dlp with cookies and returns the file path.
    """
    try:
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': 'audio.%(ext)s',
            'cookiefile': '/content/cookies (2).txt',  # Use the exported cookies
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=True)
            return "audio.mp3"
    except Exception as e:
        return f"Error downloading audio: {str(e)}"

def convert_audio_to_wav(audio_file):
    """
    Converts the downloaded MP3 audio to WAV format using pydub.
    """
    wav_file = "audio.wav"
    try:
        AudioSegment.from_mp3(audio_file).export(wav_file, format="wav")
        return wav_file
    except Exception as e:
        return f"Error converting to WAV: {str(e)}"

def transcribe_audio(audio_path, chunk_length=30):
    """
    Splits audio into smaller chunks and transcribes each chunk separately.
    Args:
        audio_path (str): Path to the audio file.
        chunk_length (int): Length of each chunk in seconds (default: 30).
    Returns:
        list: List of dictionaries containing transcribed text and timestamps.
    """
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_wav(audio_path)
    total_duration = len(audio) / 1000  # Convert to seconds
    transcribed_segments = []

    print("Transcribing audio in chunks...")

    # Split and transcribe audio in chunks
    for start in range(0, int(total_duration), chunk_length):
        end = min(start + chunk_length, int(total_duration))
        chunk = audio[start * 1000:end * 1000]  # Extract chunk in milliseconds
        chunk.export("chunk.wav", format="wav")  # Save chunk temporarily

        with sr.AudioFile("chunk.wav") as source:
            try:
                audio_data = recognizer.record(source)
                text = recognizer.recognize_google(audio_data)
                transcribed_segments.append({
                    "start": start,
                    "end": end,
                    "text": text
                })
            except sr.UnknownValueError:
                transcribed_segments.append({
                    "start": start,
                    "end": end,
                    "text": "[Unintelligible]"
                })
            except sr.RequestError as e:
                return f"Error with the speech recognition service: {str(e)}"

    os.remove("chunk.wav")  # Clean up temporary chunk file
    return transcribed_segments

def get_transcript_unlisted(video_url):
    """
    Tries to fetch the transcript using youtube_transcript_api first,
    then falls back to downloading and transcribing audio if necessary.
    """
    video_id = extract_video_id(video_url)
    if not video_id:
        return "Invalid YouTube URL."

    # Try to fetch transcript using youtube_transcript_api
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        # Add 'end' time to each segment
        for segment in transcript:
            segment["end"] = segment["start"] + segment["duration"]
        return transcript  # Return transcript with timestamps
    except:
        print("Transcript not available via API, attempting audio transcription...")

    # Download and transcribe audio if no transcript is available
    audio_file = download_audio(video_url)
    if "Error" in audio_file:
        return audio_file

    wav_file = convert_audio_to_wav(audio_file)
    if "Error" in wav_file:
        return wav_file

    transcription = transcribe_audio(wav_file)

    # Cleanup temporary files
    os.remove(audio_file)
    os.remove(wav_file)

    return transcription

def save_transcript_to_file(transcript, filename="transcript.txt"):
    """
    Saves the transcript to a text file.
    Args:
        transcript (list or str): The transcript to save.
        filename (str): The name of the output file.
    """
    with open(filename, "w", encoding="utf-8") as file:
        if isinstance(transcript, list):
            for segment in transcript:
                file.write(f"{segment['start']} - {segment['end']}: {segment['text']}\n")
        else:
            file.write(transcript)
    print(f"Transcript saved to {filename}")

# Example usage
if __name__ == "__main__":
    video_url = input("Enter the YouTube video URL: ")
    transcript = get_transcript_unlisted(video_url)

    if isinstance(transcript, list):
        print("\nTranscript with Timestamps:")
        for segment in transcript:
            print(f"{segment['start']} - {segment['end']}: {segment['text']}")
    else:
        print("\nTranscript:\n", transcript)

    # Save transcript to a text file
    save_transcript_to_file(transcript, "transcript.txt")

In [ ]:
import re
import urllib.parse
import requests
from youtube_transcript_api import YouTubeTranscriptApi
from pytube import YouTube
import speech_recognition as sr
from pydub import AudioSegment
import os
import yt_dlp

def extract_video_id(video_url):
    parsed_url = urllib.parse.urlparse(video_url)
    query_params = urllib.parse.parse_qs(parsed_url.query)

    if "v" in query_params:
        return query_params["v"][0]

    match = re.search(r"(youtu\.be/|youtube\.com/embed/|youtube\.com/shorts/)([\w-]+)", video_url)
    if match:
        return match.group(2)

    return None

def download_audio(video_url):
    try:
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': 'audio.%(ext)s',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=True)
            return "audio.mp3"
    except Exception as e:
        return f"Error: {str(e)}"

def convert_audio_to_wav(audio_file):
    if "Error" in audio_file:
        return audio_file
    wav_file = "audio.wav"
    try:
        AudioSegment.from_mp3(audio_file).export(wav_file, format="wav")
        return wav_file
    except Exception as e:
        return f"Error converting to WAV: {str(e)}"

def transcribe_audio(audio_path, chunk_length=30):
    if "Error" in audio_path:
        return audio_path
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_wav(audio_path)
    total_duration = len(audio) / 1000
    transcribed_segments = []
    formatted_transcript = {}

    for start in range(0, int(total_duration), chunk_length):
        end = min(start + chunk_length, int(total_duration))
        chunk = audio[start * 1000:end * 1000]
        chunk.export("chunk.wav", format="wav")

        with sr.AudioFile("chunk.wav") as source:
            try:
                audio_data = recognizer.record(source)
                text = recognizer.recognize_google(audio_data)
            except sr.UnknownValueError:
                text = "[Unintelligible]"
            except sr.RequestError as e:
                return f"Error with the speech recognition service: {str(e)}"

        formatted_transcript[f"{start}-{end}"] = text

    os.remove("chunk.wav")
    return formatted_transcript

def get_transcript_unlisted(video_url):
    video_id = extract_video_id(video_url)
    if not video_id:
        return "Invalid YouTube URL."

    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        formatted_transcript = {}
        for segment in transcript:
            start = int(segment["start"] // 30) * 30
            end = start + 30
            if f"{start}-{end}" not in formatted_transcript:
                formatted_transcript[f"{start}-{end}"] = ""
            formatted_transcript[f"{start}-{end}"] += " " + segment["text"]
        return formatted_transcript
    except:
        print("Transcript not available via API, attempting audio transcription...")

    audio_file = download_audio(video_url)
    if "Error" in audio_file:
        return audio_file

    wav_file = convert_audio_to_wav(audio_file)
    if "Error" in wav_file:
        return wav_file

    transcription = transcribe_audio(wav_file)
    os.remove(audio_file)
    os.remove(wav_file)

    return transcription

def save_transcript_to_file(transcript, filename="transcript.txt"):
    if isinstance(transcript, str):
        print("Error:", transcript)
        return

    with open(filename, "w", encoding="utf-8") as file:
        for time_range, text in transcript.items():
            file.write(f"{time_range}: {text}\n")
    print(f"Transcript saved to {filename}")

if __name__ == "__main__":
    video_url = input("Enter the YouTube video URL: ")
    transcript = get_transcript_unlisted(video_url)

    if isinstance(transcript, dict):
        print("\nFormatted Transcript:")
        for time_range, text in transcript.items():
            print(f"{time_range}: {text}")
    else:
        print("\nError:\n", transcript)

    save_transcript_to_file(transcript, "transcript.txt")